In [251]:
import os as os
import pandas as pd
import csv as csv

In [252]:
hero = pd.read_json("purchase_data.json", orient = "columns")
hero.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [195]:
#Total number of players
totalnumber = len(hero["SN"].unique())
#f"The total number of players is: {totalnumber}"
total_df = pd.DataFrame({
    "Total Players": [totalnumber]})
total_df


,Total Players
0,573


In [196]:
avg_price = hero["Price"].mean()

In [253]:
#Purchasing Analysis (Total)
#Total Number of Unique Items (Note: I'm using the Item ID number even though its unique value
#does not match the unique number of Item Names)

total_uniq_item = len(hero["Item ID"].unique())

#f"The total number of unique items is: {total_uniq_item}"

In [254]:
# Total Number of Purchases
total_purchase = hero.shape[0]
f"The total number of purchases = {total_purchase}"

# Total Revenue
total_revenue = hero["Price"].sum()
total_revenue

# Average Purchase Price ( can also use hero["Price"].mean())
average_price = total_revenue/total_purchase
#f"The average price is: ${average_price:.3}"

In [255]:
new_df = pd.DataFrame({
    "Number of Unique Items": [total_uniq_item],
    "Average Price": [f"${average_price:.3}"],
    "Number of Purchases": [f"{total_purchase}"],
    "Total Revenue": ['${:,}'.format(total_revenue)]
})
new_df

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.93,780,183,"$2,286.33"


In [256]:
total_unique = hero.drop_duplicates("SN")

In [257]:
# Gender Demographics

gender = total_unique["Gender"].value_counts()
gender
# gender = hero["Gender"].value_counts()
males = gender[0]
females = gender[1]
other = gender[2]

total = int(totalnumber)
gender_df = pd.DataFrame({
    "Gender": ["Male", "Female", "Other"],
    "Players Count": [males, females, other],
    "Percentage of Players": [males/total*100, females/total*100, other/total*100]
})

gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:.2f}%".format)
gender_df

# print(f"There are {gender[0]} playing this game")


,Gender,Percentage of Players,Players Count
0,Male,81.15%,465
1,Female,17.45%,100
2,Other,1.40%,8


In [258]:
# Purchasing Analysis (Gender)

#PLEASE NOTE!!!! I am using only unique user names or "SN" for gender demographics. If I don't, I end up with 633 male 
#players alone which is well over the total number of unique players

# Purchase Count

hero_female = hero.loc[hero["Gender"] == "Female", :]
hero_male = hero.loc[hero["Gender"] == "Male", :]
hero_other = hero.loc[hero["Gender"] == "Other / Non-Disclosed", :]

total_purchase = hero.shape[0]

purchases_female = hero_female.shape[0]
purchases_male = hero_male.shape[0]
purchases_other = hero_other.shape[0]

# Average Purchase Price
female_avg =hero_female["Price"].mean()
male_avg =hero_male["Price"].mean()
other_avg =hero_other["Price"].mean()

# Total Purchase Value
female_revenue = hero_female["Price"].sum()
male_revenue = hero_male["Price"].sum()
other_revenue = hero_other["Price"].sum()

# Normalized Totals
fem_norm_total = float(female_revenue/purchases_female)
m_norm_total = float(male_revenue/purchases_male)
o_norm_total = float(other_revenue/purchases_other)

#Dataframe
gender_2_df = pd.DataFrame({
    "Gender": ["Male", "Female", "Other"],
    "Purchase Count": [purchases_male, purchases_female, purchases_other],
    "Average Purchase Price": [male_avg, female_avg, other_avg],
    "Total Purchase Value": [male_revenue, female_revenue, other_revenue],
    "Normalized Totals": [m_norm_total, fem_norm_total, o_norm_total]
})

gender_2_df["Average Purchase Price"] = gender_2_df["Average Purchase Price"].map("${:.2f}".format)
gender_2_df["Normalized Totals"] = gender_2_df["Normalized Totals"].map("${:.2f}".format)
gender_2_df["Total Purchase Value"] = gender_2_df["Total Purchase Value"].map("${:.2f}".format)

gender_2_df

,Average Purchase Price,Gender,Normalized Totals,Purchase Count,Total Purchase Value
0,$2.95,Male,$2.95,633,$1867.68
1,$2.82,Female,$2.82,136,$382.91
2,$3.25,Other,$3.25,11,$35.74


In [286]:
# Age Demographics

bins = [0, 9, 14, 19, 24, 29, 34, 39, 46]
labels_years = ["below 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Create the bins in which Data will be held
# Bins are (0 < x <= 25), (25 < x <= 50), (50 < x <= 75), (75 < x <= 100)
# bins = [0, 25, 50, 75, 100]

demo = pd.cut(hero["Age"], bins, labels=labels_years)

# Place the data series into a new column inside of the DataFrame
hero["Age Group"] = demo


In [287]:
# Create a GroupBy object based upon "Age Group"
demo_group = hero.groupby("Age Group")

In [288]:
demo_loc = hero.loc[:,["Item Name", "Age Group", "Price"]]

In [341]:
# Purchase Count
#demo_age_purchase_count = item_data.groupby(["Age Group", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
#purchase_count = demo_loc.groupby(["Item Name"]).sum()["Price"].rename("Total Purchase Value")
#purchase_count = demo_loc["Age Group"].value_counts()

# # Average Purchase Price
# avg_purchase_age = demo_loc.groupeby["Price"].mean()
# avg_purchase_age
avg_purchase_age = demo_loc.groupby(["Age Group"]).mean()["Price"]
# # Total Purchase Value
# #Average price by age group multiplied by the number of purchases

total_count = demo_loc.groupby(["Age Group"]).count()["Price"]

total_value_age = purchase_count * avg_purchase_age
purchase_count

20-24       336
15-19       133
25-29       125
30-34        64
35-39        42
10-14        35
below 10     28
40+          17
Name: Age Group, dtype: int64

In [342]:

age_df = pd.DataFrame({
                       "Total": total_count, 
                       "Percentage of Players": total_count/573 * 100
                        
                      })

age_df["Percentage of Players"] = age_df["Percentage of Players"].map("{:.2f}%".format)
age_df

,Percentage of Players,Total
Age Group,,
below 10,4.89%,28
10-14,6.11%,35
15-19,23.21%,133
20-24,58.64%,336
25-29,21.82%,125
30-34,11.17%,64
35-39,7.33%,42
40+,2.97%,17


In [343]:
age_purchase_df = pd.DataFrame({
                       "Total Purchase Value": purchase_count, 
                       "Average Purchase Price": avg_purchase_age,   
                      })


age_purchase_df["Average Purchase Price"] = age_purchase_df["Average Purchase Price"].map("${:.2f}".format)
age_purchase_df["Total Purchase Value"] = age_purchase_df["Total Purchase Value"].map("${:.2f}".format)

age_purchase_df


,Average Purchase Price,Total Purchase Value
10-14,$2.77,$35.00
15-19,$2.91,$133.00
20-24,$2.91,$336.00
25-29,$2.96,$125.00
30-34,$3.08,$64.00
35-39,$2.84,$42.00
40+,$3.16,$17.00
below 10,$2.98,$28.00


In [344]:
# Top Spenders

# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

# # SN
Top_spenders_2 = hero.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")

# Top_spenders = Top_spenders_2.sort_values("Price", ascending=False)
User_avg = hero.groupby(["SN"]).mean()["Price"].rename("Average Purchase Value")
# hero["Age"] = pd.cut(hero["Age"], bins, labels=labels_years)



# # Purchase Count
user_purchase_count = hero.groupby(["SN"]).count()["Price"].rename("Number of Purchases")
# #THIS ISN"t MATCHING THE EXAMPLE...
spenders_user_data = pd.DataFrame({"Top Spenders": Top_spenders_2,
                                   "Average Purchase Value": User_avg,
                                   "Number of Purchases": user_purchase_count
})
spenders_user_data["Average Purchase Value"] = spenders_user_data["Average Purchase Value"].map("${:.2f}".format)
spenders_user_data["Top Spenders"] = spenders_user_data["Top Spenders"]
x = spenders_user_data.sort_values("Top Spenders", ascending=False)
x.head()


,Average Purchase Value,Number of Purchases,Top Spenders
SN,,,
Undirrala66,$3.41,5,17.06
Saedue76,$3.39,4,13.56
Mindimnya67,$3.18,4,12.74
Haellysu29,$4.24,3,12.73
Eoda93,$3.86,3,11.58


In [345]:
item_data = hero.loc[:, ['Item ID', 'Item Name', 'Price']]

In [346]:
total_item_purchase = item_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")

In [347]:
# Most Popular Items

# Identify the 5 most popular items by purchase count, then list (in a table):

# Item ID
# # Item Name
# # Purchase Count
# # Item Price
# # Total Purchase Value

item_data = hero.loc[:,["Item ID", "Item Name", "Price"]]
total_item_purchase = item_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
item_avg = item_data.groupby(["Item ID", "Item Name"]).mean()["Price"].rename("Total Purchase Value")
item_count = item_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Total Purchase Value")
                
most_pop_item = pd.DataFrame({"Total Purchase Value": total_item_purchase,
                              "Item Price": item_avg,
                              "Item Count": item_count
                             })

most_pop_item["Item Price"] = most_pop_item["Item Price"].map("${:.2f}".format)
most_pop_item["Total Purchase Value"] = most_pop_item["Total Purchase Value"]
most_pop_item_final = most_pop_item.sort_values("Item Count", ascending=False)
most_pop_item_final.head(5)


,,Item Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,25.85
84,Arcane Gem,11,$2.23,24.53
31,Trickster,9,$2.07,18.63
175,Woeful Adamantite Claymore,9,$1.24,11.16
13,Serenity,9,$1.49,13.41


In [348]:
most_pop_item_final["Total Purchase Value"] = pd.to_numeric(most_pop_item_final["Total Purchase Value"])

most_profit_item = most_pop_item.sort_values("Total Purchase Value", ascending=False)
#Trying to format a bit because it won't sort with the dollar sign in there. inumerate will not work for me.
# most_pop_item_final["Total Purchase Value"] = most_pop_item_final["Total Purchase Value"].map("${:.2f}".format)
# most_pop_item["Total Purchase Value"] = most_pop_item["Total Purchase Value"].map("${:.2f}".format)

most_profit_item.head(5)

,,Item Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,37.26
115,Spectral Diamond Doomblade,7,$4.25,29.75
32,Orenmir,6,$4.95,29.70
103,Singed Scalpel,6,$4.87,29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,28.88
